# Module 4: Data Sharing and Next Steps.


In this section we will examine how to align assembled genomes, perform phylogenetic analysis and interpret phylogenetic trees for outbreak investigations.


As you have perfomed before:

1.) Click on File on the top left corner and select save a copy in drive
Your changes will not be saved if you do not do this step.

2.) Click on the name of the workbook in the top left corner and replace "Copy of" with your full name.

You will be submitting the downloaded notebook file as your proof of completion for this module.

# **Part 1**: Phylogenetics and interpretation

## Let's install analysis packages needed.

For this portion we will use:  
**seqtk** for assess genome quality (https://github.com/lh3/seqtk)  
**Mafft** for genome alignment (https://mafft.cbrc.jp/alignment/software/)  
**snp-site** for quick SNP difference assessment (https://github.com/sanger-pathogens/snp-sites)  
**Fasttree** for phylogenetic tree building (http://www.microbesonline.org/fasttree/)  
**Phylo** from biopython for quick tree visualisation (https://biopython.org/wiki/Phylo). *Note*: there are lots of tree visualisaiton programmes, most commonly used are ggtree(R), ete3 (python) and itol (https://itol.embl.de/).  

microreact account (https://microreact.org/), which you can setup/sign in with your google account.

In [ ]:
!python --version

Python 3.7.15


In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:24
🔁 Restarting kernel...


In [ ]:
!conda install -c conda-forge biopython

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ failed with initial frozen solve. Retrying with flexible solve.
Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / 

In [ ]:
!conda install -c bioconda mafft snp-sites fasttree seqtk snp-dists


Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - fasttree
    - mafft
    - seqtk
    - snp-dists
    - snp-sites


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-22.9.0               |   py37h89c1867_1         960 KB  conda-forge
    fasttree-2.1.11            |       hec16e2b_1         252 KB  bioconda
    libzlib-1.2.13             |       h166bdaf_4          64 KB  conda-forge
    llvm-openmp-8.0.1          |       hc9558a2_0         439 KB  conda-forge
    mafft-7.508                |       hec16e2b_0         3.1 MB  bioconda
    openmp-8.0.1               |               

## Download dataset to be analysed.

There are three datasets associated with this module. First two datasets are examples in the instruction slides. The last datasset is assemsment dataset. Analysing the example datasets alongside the instruction videos will give you the experience needed to analyse the assesment dataset.

In [ ]:
!wget https://wcs_data_transfer.cog.sanger.ac.uk/Module4_data_zip.zip
!unzip Module4_data_zip.zip

--2022-11-21 15:29:29--  https://wcs_data_transfer.cog.sanger.ac.uk/Module4_data_zip.zip
Resolving wcs_data_transfer.cog.sanger.ac.uk (wcs_data_transfer.cog.sanger.ac.uk)... 193.62.203.61, 193.62.203.62, 193.62.203.63
Connecting to wcs_data_transfer.cog.sanger.ac.uk (wcs_data_transfer.cog.sanger.ac.uk)|193.62.203.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 480131 (469K) [application/zip]
Saving to: ‘Module4_data_zip.zip’

Module4_data_zip.zi 100%[===================>] 468.88K   380KB/s    in 1.2s    

2022-11-21 15:29:32 (380 KB/s) - ‘Module4_data_zip.zip’ saved [480131/480131]

Archive:  Module4_data_zip.zip
  inflating: Module4_data/paper_link.docx  
  inflating: Module4_data/dataset_1_flight/20CV0408.fasta  
  inflating: Module4_data/dataset_1_flight/20CV0415.fasta  
  inflating: Module4_data/dataset_1_flight/20CV0409.fasta  
  inflating: Module4_data/ENA submission data/CaseA_manifest.txt  
  inflating: Module4_data/dataset_1_flight/20CV0401.fasta  

### Dataset 1

First we will assess genome quality to see if the genomes are good enough for analysis. For SARs_CoV2, we are mainly interested in the number of Ns. The package we will use is seqtk comp, which gives a lots of statistics for a given sequence file in fasta format.   

Output format: chr, length, #A, #C, #G, #T, #2, #3, #4, #CpG, #tv, #ts, #CpG-ts

9th column, #4 is the number of Ns, 4 ambiguities, ie A, G, T, or C.

If you have separated fasta files, it might be easier to combine the files together.  Otherwise, you have to go through the genome files separately.

In [ ]:
#cat (concatenate) command combine the fastas into one file
#seqtk comp analysis the sequence information
#cut -f 1,9 selects the column 1 and 9, the information we need.
!cat Module4_data/dataset_1_flight/20CV*.fasta > dataset_1_flight_combined.fasta
!seqtk comp dataset_1_flight_combined.fasta | cut -f 1,9 > dataset_1_quality.tsv

In [ ]:
!cat dataset_1_quality.tsv

hCoV-19/NewZealand/20CV0398/2020	0
hCoV-19/NewZealand/20CV0401/2020	1
hCoV-19/NewZealand/20CV0408/2020	1
hCoV-19/NewZealand/20CV0409/2020	1
hCoV-19/NewZealand/20CV0410/2020	1140
hCoV-19/NewZealand/20CV0414/2020	0
hCoV-19/NewZealand/20CV0415/2020	1


Quality looks good, 20CV0410 is not as good as the rest. It  has 1140 ambiguous bases but it is good enough for analysis. In general, we require 90%, or fewer than **3000** Ns the genome to be assembled for phylogenetic tree analysis. However, there are groups that has less stringent requirrments, some go as low as 50%. Having 70% of the genome assembled is the default in pangolin for lineage assignment.

### alignment

We will use mafft for alignment. It is fairly fast and pretty accurate.

There are many options for alignning sequences in mafft. --auto is a good option where the programme itself chooses the most efficient (good balance between speed and accuracy) algorithm. Alignment could take days to align long sequences if using the most accurate algorthm.If you are aligning short sequences, such as a region of the spike gene/protein, you can use more accurate options. The mafft website has good examples of what to use when.

For mafft to align the sequences, in our case whole genomes, you need to combine the sequnence you want to align into one file. Good thing is you have done that already and since all genomes are good we don't need to exclude any from the alignment. 

In [ ]:
!mafft --auto dataset_1_flight_combined.fasta > dataset_1_flight_aligned.fasta

nthread = 0
nthreadpair = 0
nthreadtb = 0
ppenalty_ex = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..

There are 1144 ambiguous characters.
    1 / 7
done.

Constructing a UPGMA tree (efffree=0) ... 
    0 / 7
done.

Progressive alignment 1/2... 
STEP     6 / 6  f
done.

Making a distance matrix from msa.. 
    0 / 7
done.

Constructing a UPGMA tree (efffree=1) ... 
    0 / 7
done.

Progressive alignment 2/2... 
STEP     6 / 6  f
done.

disttbfast (nuc) Version 7.508
alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0
0 thread(s)


Strategy:
 FFT-NS-2 (Fast but rough)
 Progressive method (guide trees were built 2 times.)

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft --man' and the mafft page.

The default gap scoring scheme has been changed in version 7.110 (2013 Oct).
It tends to insert more 

### quick look at the genome differences
snp-site is a really good software to give you an idea how closely related your genomes of interest are. For small datasets such as this, this is really good. It analyses your alignment and give you a SNP alignmern as default. it can also give you snp information in VCF format.

In [ ]:
!snp-sites dataset_1_flight_aligned.fasta

>hCoV-19/NewZealand/20CV0398/2020
G
>hCoV-19/NewZealand/20CV0401/2020
T
>hCoV-19/NewZealand/20CV0408/2020
G
>hCoV-19/NewZealand/20CV0409/2020
G
>hCoV-19/NewZealand/20CV0410/2020
G
>hCoV-19/NewZealand/20CV0414/2020
G
>hCoV-19/NewZealand/20CV0415/2020
G


from this quick look, you will see most of genomes are indistinguishable, apart from 20CV0401 which has an additional SNP. 

### interpretation

what can we say about a bunch of identical genomes?  

Since we know these people came on the same flight, we can analyse one of the genomes using https://genome.ucsc.edu/cgi-bin/hgPhyloPlace or if you have an account you can use the GISAID audacity programme to find the closest international genome. Download a genome and try this with your web browser.

see publication https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7920679/ for the analysis of this dataset.

In [ ]:
!cat Module4_data/dataset_1_flight/dataset_1_flight_metadata.tsv

Case	Accession	GISAIDID	PreflightTestinResult	PreflightTestinDate	SymtomOnset	DatePositveTest	CT	CountryOfOrigin	LayoverTime	FlightSeat	BusToQuranteenHotel
A	20CV0408	EPI_ISL_582019	Negative	2020-09-24	2020-10-01	2020-10-02	16.40	Switzerland	9h27min	26G	Bus1
B	20CV0409	EPI_ISL_582020	Negative	2020-09-24	2020-10-02	2020-10-02	29.30	Switzerland	9h27min	26D	Bus1
C	20CV0410	EPI_ISL_582021	Negative	2020-09-25	Asymptomatic	2020-10-02	36.80	Ukraine	11h30min	24C	"Bus1briefly,transportedonbus2"
D	20CV0401	EPI_ISL_582018	Negative	2020-09-24	2020-10-04	2020-10-07	20.40	Ireland	8h18min	27D	Bus1
E	20CV0398	EPI_ISL_582017	NotTested	NA	Asymptomatic	2020-10-06	22.30	India	70h54min	28G	Bus3
F	20CV0414	EPI_ISL_582022	Negative	2020-09-25	2020-10-03	2020-10-08	22.30	SouthAfrica	5h44min	24D/E/F/G	Bus2
G	20CV0415	EPI_ISL_582023	NotTested	NA	2020-10-09	2020-10-08	19.10	SouthAfrica	5h44min	24D/E/F/G	Bus2


### Dataset 2

In [ ]:
!seqtk comp Module4_data/dataset_2_hotel/dataset_2_quarantineHotel.fasta | cut -f 1,9 > dataset_2_quality.tsv
!cat dataset_2_quality.tsv
#are the genome assemblies of good quality?

In [ ]:
#fasta files are already concatenated.
!mafft --auto Module4_data/dataset_2_hotel/dataset_2_quarantineHotel.fasta > dataset_2_quarantineHotel_aligned.fasta

In [ ]:
!snp-sites dataset_2_quarantineHotel_aligned.fasta

In [ ]:
#can look at snp distance between pairs of samples using snp-dist
!snp-dists dataset_2_quarantineHotel_aligned.fasta


The alignment is a little bit more complicated with more differences, we need a phylogenetic tree to visualise the relationships

In [ ]:
!fasttree -nt -gtr -gamma dataset_2_quarantineHotel_aligned.fasta > dataset_2_quarantineHotel_aligned.nwk

In [ ]:
!cat dataset_2_quarantineHotel_aligned.nwk
#as you can see it is quite difficult to interpret this format without visualising in tree-form


In [ ]:
from Bio import Phylo

tree = Phylo.read("dataset_2_quarantineHotel_aligned.nwk", "newick")
print("tree")
Phylo.draw_ascii(tree)


interpretation

This dataset is about an incursion and onward spread into the community from quarantine hotel. What can we say about that tree? Follow the publication and instruction video. Metadata associated with the case in dataset_2_hotel/dataset_2_quarantineHotel_metadata.tsv. 


see publication https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8084504/ for the analysis of this dataset.

Compare the above tree to the publication, how come it looks different?

In [ ]:
!cat Module4_data/dataset_2_hotel/dataset_2_quarantineHotel_metadata.tsv

In [ ]:
#let's add the 2020 reference genome to the dataset. 
!mafft --auto --add Module4_data/dataset_2_hotel/dataset_2_quarantineHotel.fasta Module4_data/dataset_2_hotel/references.fasta > dataset_2_quarantineHotel_aligned_ref.fasta

In [ ]:
!fasttree -nt -gtr -gamma dataset_2_quarantineHotel_aligned_ref.fasta > dataset_2_quarantineHotel_aligned_ref.nwk

Now if we root the phylogeny using the reference, the tree looks more like the publication. How come?  

Have a look at the snp alignment as well using snp-site. You will see that using the reference allowed the identification of ancestral bases, so now we know what nucleotide changes are mutations from wildtype. Previously the tree algorithm made the assumption that the majority base is the ancestral sequence, which is not correct.

In [ ]:
from Bio import Phylo

treeOutgroup = Phylo.read("dataset_2_quarantineHotel_aligned_ref.nwk", "newick")
treeOutgroup.root_with_outgroup({"name": "nCoV2019|Wuhan-Hu-1|MN908947|China|Wuhan|2019-12"}) 
print("treeOutgroup")
Phylo.draw_ascii(treeOutgroup)

## watch micoreact and nextstrain demo/video

looking at trees this way is pretty difficult, there are some good tools out there that allows interactive trees and metadate visualisation.

https://docs.microreact.org/   
https://nextstrain.org/community/narratives/ESR-NZ/GenomicsNarrativeSARSCoV2/aotearoa-border-incursions

Demo microreact project of dataset 2   
https://microreact.org/project/5ELv2rXSKKeZ8XZCFXq9Ug-dataset2hotel


### Other tools for larger datasets and tree building
mafft even at the minimum accuracy model can be quite slow once you have 1000s of genomes to align. To speed things up, you can add new genomes to existing alignments or divide genomes into groups such as lineages or sublineages and then merge the alignments. The mafft website has good explaination of these tricks and their issues.    

nextalign is another tool you can use. It is super fast! (https://docs.nextstrain.org/projects/nextclade/en/stable/user/nextalign-cli.html)  

igtree2 is a comprehensive (not Baysian) phylogeny building package that can test for best substition models for your dataset, perform bootstrap analysis, compare phylogeny etc etc. It is probably one of the best tree building tools (that is reasnabley fast) out there currently. (http://www.iqtree.org/)

usher is a software package that allows adding sequences to existing trees without doing all the alignment again. (https://www.nature.com/articles/s41588-021-00862-7) We used this when we analysed the flight dataset. There are some caveats and issues you need to be aware of. If the existing alignment is really clean and the sequences to be added are high-quality, using usher is a good fast way to get some quick preliminary results. However, it does not deal with Ns or gaps very well.  

For timed baysian phyolgeny, BEAST2 is often used. (https://www.beast2.org/) Here is a nice review. https://www.nature.com/articles/s41559-017-0280-x  

### Now we are ready for the assignment 

Cells below will be checked.

For this assignment you will investigate a potential outbreak at a health care facility. The genomes included are from cases that are around the time of the outbreak and all has some links to the health care facility. Some cases have had clear contacts with other known cases (Epi-links to Outbreak:yes) and for some contacts have not been found (Epi-links to Outbreak:not found). And some cases are unknown (Epi-links to Outbreak: unknown).  Your job is to use the tree and report date to decide whether those cases labeld unknown should be investigated further, if they are genomically linked to the outbreak or can be excluded from the outbreak.

You need to:  
1) QC the genomes given and exclude any poor quality genomes from the analysis  
2) Align the good quality genomes (**<3000 Ns**) and construct a phylogenetic tree  
3) Use the tree you generated and metadata given to set up a microreact project for visualisation and examination of the tree  
4) Using the metadata given to decide whether isolates under question are in the outbreak   
5) Answer questions in the assesment  

**hints**  

you can use seqtk subseq to remove low quality sequences, or you can move (mv) the low quality sequences into another directory.

Use snp distance to see if some of the cases are "too" different than the rest.  

In [ ]:
#Quality control the genome assemblies to see if any of them are not suitable for further analysis.


In [ ]:
#list the isolates that you think maynot be suitable for futhure analysis below
#change XXX to isolate number
!echo "these isolates does not meet minimum quality standards: XXX and XXX"

In [ ]:
#align the good quality genomes with mafft


In [ ]:
#build a phylogenetic tree with the mafft alignment


In [ ]:
#visualise the tree with Phylo
#remember to "from Bio import Phylo" if you haven't done it already


Now you have all the files needed to creat a microreact project.  
Download your tree and the metadata file: outbreak_microreact.tsv

**Upload a screen grab of your microreact project in the cell below**  
You can do this by double click on the cell  below  
then click on the insert image icon, black with white mountain and then select the iamge file.  Depends on the picture it might be a bit slow.  
The picture should be saved once you save your notebook.


**insert your microreact screen grab here**  
double click here.

**now you have the micreact project it will make it easier for you to answer the assessment questions**

Optional use snp-dist and snp-site to help you with the analysis

**save your notebook now**

## **Part 2** Data sharing

Please follow the talk and slides in **Sections 5, 6 and 7** for instructions.  